In [126]:
def Strategy_1():
    import tushare as ts
    import os
    import datetime
    import pymongo
    import pandas

    client = pymongo.MongoClient("127.0.0.1", 27017)
    
    startDate = datetime.datetime.now() + datetime.timedelta(days=-60)
    stocks = client.Common.Stock.find({"foundTime": {"$lt": startDate}})
    threshold = 9.9
    today = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
    found = []
    minDays = 1000

    for item in stocks:
        code = item["code"]
        collection = client.Trade[code]
        res = list(collection.find({}, limit=4, sort=[("date", pymongo.DESCENDING)]))
        if len(res) < 4:
            continue
        changes = []
        for i in range(3):
            if res[i+1]["close"] == 0:
                d = 0
            else:
                d = (res[i]["close"] / res[i+1]["close"] - 1) * 100
            changes.append(d)
        if changes[0] < threshold or (changes[1] <= threshold and changes[2] <= threshold):
            continue
        days = (today - res[3]["date"]).days
        if days > minDays:
            continue
        if days < minDays:
            minDays = days
            found = []
        found.append({
            "code": item["code"],
            "name": item["name"],
            "changes": changes
        })

    if len(found) > 0:
        print pandas.DataFrame(found, columns=["code", "name", "changes"])
    else:
        print("No Result")

In [128]:
Strategy_1()

     code  name                                        changes
0  000816  智慧农业  [10.1307189542, 10.071942446, -4.13793103448]
1  300077  国民技术           [9.99039385207, 10.0422832981, 10.0]
2  300104   乐视网       [9.9609375, 10.1075268817, 2.1978021978]
3  002806  华锋股份            [10.0, 10.0094428706, 10.025974026]
